***Image Stiching Different Methods:***


***1) Feature-based Image Stitching using OpenCV's Stitcher class***


***It Detects keypoints in each image using feature detectors like ORB, SIFT, SURF, etc and Matches features between overlapping images using KNN (k-nearest neighbors) + RANSAC to remove outliers. It Warps images into alignment using the homography and blends them into a panorama.***


In [2]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import cv2
import os
import time

# === Configuration ===
SAVE_DIR = 'captured_frames'
FPS = 2  # frames per second to capture
CAM_INDEX = 0  # default camera

# === Setup ===
if not os.path.exists(SAVE_DIR):
    os.makedirs(SAVE_DIR)

cap = cv2.VideoCapture(CAM_INDEX)
if not cap.isOpened():
    print("Error: Could not open camera.")
    exit()

collect_frames = False
last_capture_time = 0
captured_images = []

print("Press 'i' to start/stop capturing, 'q' to quit.")

frame_counter = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    cv2.imshow("Live Feed", frame)

    key = cv2.waitKey(1) & 0xFF

    # Start/Stop frame collection with 'i'
    if key == ord('i'):
        collect_frames = not collect_frames
        print("Capturing:" if collect_frames else "Stopped Capturing")
        time.sleep(0.5)  # debounce

    # Quit with 'q'
    if key == ord('q'):
        break

    # Capture frames if enabled and based on FPS
    if collect_frames:
        current_time = time.time()
        if current_time - last_capture_time >= 1 / FPS:
            frame_filename = f"{SAVE_DIR}/frame_{frame_counter}.jpg"
            cv2.imwrite(frame_filename, frame)
            captured_images.append(frame_filename)
            print(f"Captured: {frame_filename}")
            frame_counter += 1
            last_capture_time = current_time

cap.release()
cv2.destroyAllWindows()

# === Perform Stitching ===
print("Stitching images...")

images = [cv2.imread(img_path) for img_path in captured_images]
stitcher = cv2.Stitcher_create()

(status, stitched) = stitcher.stitch(images)

if status == cv2.Stitcher_OK:
    print("Stitching successful.")
    cv2.imshow("Stitched Image", stitched)
    cv2.imwrite("stitched_output.jpg", stitched)
    cv2.waitKey(0)
else:
    print("Stitching failed. Error code:", status)

cv2.destroyAllWindows()


***2) Phase Correlation Method***


In [5]:
import cv2
import os
import time
import numpy as np

# === Configuration ===
SAVE_DIR = 'captured_frames'
FPS = 2  # frames per second to capture
CAM_INDEX = 0  # default camera

# === Setup ===
if not os.path.exists(SAVE_DIR):
    os.makedirs(SAVE_DIR)

cap = cv2.VideoCapture(CAM_INDEX)
if not cap.isOpened():
    print("Error: Could not open camera.")
    exit()

collect_frames = False
last_capture_time = 0
captured_images = []

print("Press 'i' to start/stop capturing, 'q' to quit.")

frame_counter = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    cv2.imshow("Live Feed", frame)

    key = cv2.waitKey(1) & 0xFF

    # Start/Stop frame collection with 'i'
    if key == ord('i'):
        collect_frames = not collect_frames
        print("Capturing:" if collect_frames else "Stopped Capturing")
        time.sleep(0.5)  # debounce

    # Quit with 'q'
    if key == ord('q'):
        break

    # Capture frames if enabled and based on FPS
    if collect_frames:
        current_time = time.time()
        if current_time - last_capture_time >= 1 / FPS:
            frame_filename = f"{SAVE_DIR}/frame_{frame_counter}.jpg"
            cv2.imwrite(frame_filename, frame)
            captured_images.append(frame_filename)
            print(f"Captured: {frame_filename}")
            frame_counter += 1
            last_capture_time = current_time

cap.release()
cv2.destroyAllWindows()

# === Phase Correlation Stitching ===
def phase_correlation_stitch(img1, img2):
    gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

    # Use Hanning window to reduce spectral leakage
    hann = cv2.createHanningWindow(gray1.shape[::-1], cv2.CV_64F)
    shift, _ = cv2.phaseCorrelate(np.float64(gray2), np.float64(gray1), hann)

    dx, dy = int(round(shift[0])), int(round(shift[1]))
    print(f"Estimated shift: dx={dx}, dy={dy}")

    height = max(img1.shape[0], img2.shape[0] + abs(dy))
    width = img1.shape[1] + img2.shape[1] + abs(dx)

    stitched = np.zeros((height, width, 3), dtype=np.uint8)
    stitched[:img2.shape[0], :img2.shape[1]] = img2

    x_offset = img2.shape[1] + dx
    y_offset = dy if dy > 0 else 0

    stitched[y_offset:y_offset+img1.shape[0], x_offset:x_offset+img1.shape[1]] = img1

    return stitched

# === Stitch captured images using Phase Correlation ===
print("Stitching images using Phase Correlation...")

if len(captured_images) < 2:
    print("Not enough images to stitch.")
    exit()

stitched = cv2.imread(captured_images[0])
for img_path in captured_images[1:]:
    next_img = cv2.imread(img_path)
    stitched = phase_correlation_stitch(next_img, stitched)

cv2.imshow("Phase Correlation Stitched Image", stitched)
cv2.imwrite("stitched_output_phase.jpg", stitched)
cv2.waitKey(0)
cv2.destroyAllWindows()


Press 'i' to start/stop capturing, 'q' to quit.
Capturing:
Captured: captured_frames/frame_0.jpg
Captured: captured_frames/frame_1.jpg
Captured: captured_frames/frame_2.jpg
Captured: captured_frames/frame_3.jpg
Captured: captured_frames/frame_4.jpg
Captured: captured_frames/frame_5.jpg
Captured: captured_frames/frame_6.jpg
Captured: captured_frames/frame_7.jpg
Captured: captured_frames/frame_8.jpg
Captured: captured_frames/frame_9.jpg
Captured: captured_frames/frame_10.jpg
Captured: captured_frames/frame_11.jpg
Captured: captured_frames/frame_12.jpg
Captured: captured_frames/frame_13.jpg
Captured: captured_frames/frame_14.jpg
Captured: captured_frames/frame_15.jpg
Stitching images using Phase Correlation...
Estimated shift: dx=-14, dy=8


error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\phasecorr.cpp:528: error: (-215:Assertion failed) src1.size == src2.size in function 'cv::phaseCorrelate'
